In [1]:
import sys
# from product import *
# from data_util.product import *
from data_util.mainCat import *

import pandas as pd
import random
import time
import re

import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk

import sys
import shutil
import tqdm
import random

from copy import deepcopy
# from product import *

VOCAB_SIZE = 50000
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

# Key word Attention DataSet 讀取

In [2]:
# main_cat = All_Electronics().getAttr()
main_cat = Pet_Supplies().getAttr()
# main_cat = Sports_Outdoors().getAttr()
# main_cat = Health_personal_Care().getAttr()

xlsx_path = "XLSX/main_cat/%s_key.xlsx"%(main_cat)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
# key_train_df.to_excel(csv_path, encoding='utf8')
orign_key_df = pd.read_excel(xlsx_path)
print(xlsx_path + " Read finished")
len(orign_key_df)
orign_key_df.head()

XLSX/main_cat/Pet Supplies_key.xlsx Read finished


,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features
0,1403740800,it is not clear on the description but this br...,be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,['it is not clear on the description but this ...,<s> be aware of modification need </s> \n,101,7,3,4,axle modify bracket unusable,it is necessary to modify the axle for that an...,hand axle bracket blaster metro rest handle de...
1,1428278400,a lot to pay for the wall bracket but it works...,a lot to pay for the wall bracket but .,Appliances,Pet Supplies,Parts & Accessories,['a lot to pay for the wall bracket but it wor...,<s> a lot to pay for the wall bracket but </s> \n,69,11,4,2,NaN,NaN,pay wall lot bracket note mount fit bracket bl...
2,1394841600,does what it is supposed to but ridiculously o...,okay but ridiculous price,Appliances,Pet Supplies,Parts & Accessories,['does what it is suppose to but ridiculously ...,<s> okay but ridiculous price </s> \n,41,6,3,8,"piece simple,metal bent",does what it is suppose to but ridiculously ov...,piece simple suppose price metal kind include ...
3,1410739200,worked well. i use to do water changes and was...,worked well. i use to do water changes and .,Appliances,Pet Supplies,Water Filters,"['work well .', 'i use to do water change and ...",<s> work well </s>\n<s> i use to do water chan...,21,13,5,2,"pump hold,suction surprise",i use to do water change and was surprise the ...,pump water surprise cup change side glass suct...
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,['this ribbon suck it tear easily and it is my...,<s> this ribbon suck it tear easily and it is ...,97,14,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...


# Key word load

In [3]:
fn = 'FOP-View/%s_keywords2.txt' % (main_cat)
print('load %s keywords...' % (fn))
total_keywords = set()
with open(fn, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        k, v = line.split(":")
        total_keywords.add(k)

load FOP-View/Pet Supplies_keywords2.txt keywords...


# Total Opinion

In [4]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
print("total-words 已取得")

negative-words.txt
positive-words.txt
total-words 已取得


# Summary 資料清理

In [5]:
import spacy
from collections import Counter
import en_core_web_sm
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = en_core_web_sm.load()

In [6]:
def compose_summary(x):
    x = x = x.replace(".\n"," . ").replace("\n."," . ").replace("\n"," ").replace("</s>","").replace("<s>","")
    x = "<s>" + x + "</s>"  
    x = " ".join([str(token) for token in nlp(x) if (" " not in str(token)) and \
                  (str(token).isalpha()) and \
                  (len(str(token)) > 1)   ])
    return x


def create_custom_tokenizer(nlp):
    prefix_re = re.compile(r'[0-9]\.')
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search)

def calc_summary_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))

nlp.tokenizer = create_custom_tokenizer(nlp)

orign_key_df['lemm_summary'] = orign_key_df['lemm_summary'].apply(compose_summary)
orign_key_df['lemm_summary_len'] = orign_key_df['lemm_summary'].apply(calc_summary_len)


amount = len(orign_key_df)
print('Total data : %s'%(amount))

orign_key_df.head()


Total data : 547381


,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features
0,1403740800,it is not clear on the description but this br...,be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,['it is not clear on the description but this ...,be aware of modification need,101,5,3,4,axle modify bracket unusable,it is necessary to modify the axle for that an...,hand axle bracket blaster metro rest handle de...
1,1428278400,a lot to pay for the wall bracket but it works...,a lot to pay for the wall bracket but .,Appliances,Pet Supplies,Parts & Accessories,['a lot to pay for the wall bracket but it wor...,lot to pay for the wall bracket but,69,8,4,2,NaN,NaN,pay wall lot bracket note mount fit bracket bl...
2,1394841600,does what it is supposed to but ridiculously o...,okay but ridiculous price,Appliances,Pet Supplies,Parts & Accessories,['does what it is suppose to but ridiculously ...,okay but ridiculous price,41,4,3,8,"piece simple,metal bent",does what it is suppose to but ridiculously ov...,piece simple suppose price metal kind include ...
3,1410739200,worked well. i use to do water changes and was...,worked well. i use to do water changes and .,Appliances,Pet Supplies,Water Filters,"['work well .', 'i use to do water change and ...",work well use to do water change and,21,8,5,2,"pump hold,suction surprise",i use to do water change and was surprise the ...,pump water surprise cup change side glass suct...
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,['this ribbon suck it tear easily and it is my...,this ribbon suck it tear easily and it is my f...,97,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...


# review 多句合併

In [7]:
from copy import deepcopy
def compose_review(x):
    x = eval(x)
    x = "\n".join(x)
    x = x.replace(".\n"," . ").replace("\n."," . ").replace("\n"," ")    
#     tokens = [str(token) for token in x.split(" ") if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token)) ]

    return " ".join(tokens)

def calc_review_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))
key_df = deepcopy(orign_key_df)
key_df['lemm_review'] = key_df['lemm_review'].apply(compose_review)
key_df['lemm_review_len'] = key_df['lemm_review'].apply(calc_review_len)
key_df.head()

,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features
0,1403740800,it is not clear on the description but this br...,be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,it is not clear on the description but this br...,be aware of modification need,112,5,3,4,axle modify bracket unusable,it is necessary to modify the axle for that an...,hand axle bracket blaster metro rest handle de...
1,1428278400,a lot to pay for the wall bracket but it works...,a lot to pay for the wall bracket but .,Appliances,Pet Supplies,Parts & Accessories,a lot to pay for the wall bracket but it work ...,lot to pay for the wall bracket but,74,8,4,2,NaN,NaN,pay wall lot bracket note mount fit bracket bl...
2,1394841600,does what it is supposed to but ridiculously o...,okay but ridiculous price,Appliances,Pet Supplies,Parts & Accessories,does what it is suppose to but ridiculously ov...,okay but ridiculous price,44,4,3,8,"piece simple,metal bent",does what it is suppose to but ridiculously ov...,piece simple suppose price metal kind include ...
3,1410739200,worked well. i use to do water changes and was...,worked well. i use to do water changes and .,Appliances,Pet Supplies,Water Filters,work well . i use to do water change and was ...,work well use to do water change and,24,8,5,2,"pump hold,suction surprise",i use to do water change and was surprise the ...,pump water surprise cup change side glass suct...
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck it tear easily and it is my f...,this ribbon suck it tear easily and it is my f...,106,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...


In [8]:
orign_key_df.head()

,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features
0,1403740800,it is not clear on the description but this br...,be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,['it is not clear on the description but this ...,be aware of modification need,101,5,3,4,axle modify bracket unusable,it is necessary to modify the axle for that an...,hand axle bracket blaster metro rest handle de...
1,1428278400,a lot to pay for the wall bracket but it works...,a lot to pay for the wall bracket but .,Appliances,Pet Supplies,Parts & Accessories,['a lot to pay for the wall bracket but it wor...,lot to pay for the wall bracket but,69,8,4,2,NaN,NaN,pay wall lot bracket note mount fit bracket bl...
2,1394841600,does what it is supposed to but ridiculously o...,okay but ridiculous price,Appliances,Pet Supplies,Parts & Accessories,['does what it is suppose to but ridiculously ...,okay but ridiculous price,41,4,3,8,"piece simple,metal bent",does what it is suppose to but ridiculously ov...,piece simple suppose price metal kind include ...
3,1410739200,worked well. i use to do water changes and was...,worked well. i use to do water changes and .,Appliances,Pet Supplies,Water Filters,"['work well .', 'i use to do water change and ...",work well use to do water change and,21,8,5,2,"pump hold,suction surprise",i use to do water change and was surprise the ...,pump water surprise cup change side glass suct...
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,['this ribbon suck it tear easily and it is my...,this ribbon suck it tear easily and it is my f...,97,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...


# 過濾不合適的訓練資料

In [9]:
def to_words(text):
    keywords = set()
    for words in text.split(","):
        for word in words.split(" "):
            keywords.add(word)
    keywords = " ".join(keywords)
    return keywords

def calc_keyword_num(x):
    return len(x.split(" "))

# and(key_df.lemm_review_len>20)
flit_key_df = key_df[(key_df.lemm_summary_len>=4) ] # 過濾single word summary
flit_key_df = flit_key_df[(flit_key_df.lemm_review_len <= 1000) ] # 過濾single word summary
flit_key_df = flit_key_df[(flit_key_df.lemm_review_len >= 50) ] # 過濾single word summary

flit_key_df = flit_key_df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )


# FOP_keywords 資料整理

In [10]:
flit_key_df['FOP_keywords'] = flit_key_df['total_keyword']
flit_key_df['FOP_keywords'] = flit_key_df['FOP_keywords'].apply(to_words)
flit_key_df['FOP_keywords_num'] = flit_key_df['FOP_keywords'].apply(calc_keyword_num)
flit_key_df = flit_key_df[(flit_key_df.FOP_keywords_num>=2) ] # 過濾single word summary
flit_key_df.head()

,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,FOP_keywords,FOP_keywords_num
0,1403740800,it is not clear on the description but this br...,be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,it is not clear on the description but this br...,be aware of modification need,112,5,3,4,axle modify bracket unusable,it is necessary to modify the axle for that an...,hand axle bracket blaster metro rest handle de...,unusable modify axle bracket,4
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck it tear easily and it is my f...,this ribbon suck it tear easily and it is my f...,106,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...,tie tight,2
7,1365379200,my son was making his own set of chain mail ar...,it is a real steel at this price.,"Arts, Crafts & Sewing",Pet Supplies,Wire,my son was make his own set of chain mail armo...,it is real steel at this price,65,7,5,6,"store local exhibit good,consistency good,cons...",my son was make his own set of chain mail armo...,armor mail son spool store chain set stuff exp...,exhibit consistency tarnish quality good store...,7
8,1390953600,this is a good product for the price. the wire...,good product for price.,"Arts, Crafts & Sewing",Pet Supplies,Wire,this is a good product for the price . the wi...,good product for price,115,4,4,13,"product good process really manual,process man...",this is a good product for the price .\nalso t...,price product arrive weather wire spool box pr...,american guage really disclaimer manual standa...,13
9,1368662400,my son asked for this wire as a christmas gift...,great for making chainmail,"Arts, Crafts & Sewing",Pet Supplies,Wire,my son ask for this wire as a christmas gift a...,great for make chainmail,84,4,5,11,"sister dad wire great,starter great",my son ask for this wire as a christmas gift a...,item mail son dad chain gift sister wire start...,starter great wire sister dad,5


# Cheat Processing

In [11]:
flit_key_df['Cheat'] = False 

# flit_key_df.head()
from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        rev_tokens = set(row['lemm_review'].split(" "))
#         rev_tokens = [str(token) for token in rev_tokens if not (str(token) == '.' or str(token).isalpha())]
#         rev_tokens = set(rev_tokens)
        
#         if 's' in rev_tokens: rev_tokens.remove('s')
#         if '.' in rev_tokens: rev_tokens.remove('.')
#         if 'i' in rev_tokens: rev_tokens.remove('i')
#         if 'a' in rev_tokens: rev_tokens.remove('a')
#         if 'the' in rev_tokens: rev_tokens.remove('the')
        
        summ_tokens = set(row['lemm_summary'].split(" "))
        key_words = rev_tokens & summ_tokens & (total_keywords| set(opinion_lexicon["total-words"]))
        if len(key_words) > 2: 
#             print(True)
            flit_key_df.loc[i,'Cheat'] = True

#         print(rev_tokens)
#         print(summ_tokens)
#         print([len(t) for t in summ_tokens])
        pbar.update(1)
    
flit_key_df = flit_key_df[(flit_key_df.Cheat == True) ] # 過濾single word summary
amount = len(flit_key_df)
print('Total data : %s'%(amount))
flit_key_df.head()


100%|██████████| 241939/241939 [11:41<00:00, 344.80it/s]

Total data : 104632


,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,FOP_keywords,FOP_keywords_num,Cheat
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck it tear easily and it is my f...,this ribbon suck it tear easily and it is my f...,106,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...,tie tight,2,True
8,1390953600,this is a good product for the price. the wire...,good product for price.,"Arts, Crafts & Sewing",Pet Supplies,Wire,this is a good product for the price . the wi...,good product for price,115,4,4,13,"product good process really manual,process man...",this is a good product for the price .\nalso t...,price product arrive weather wire spool box pr...,american guage really disclaimer manual standa...,13,True
26,1453593600,the snaps work well. they are pretty sticky at...,they are pretty sticky at first and require so...,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the snap work well . they are pretty sticky a...,they are pretty sticky at first and require so...,63,12,3,2,"snap white,snap female",problem though 2 of my pack of 10 white snap h...,snap require snap include female pack snap whi...,white snap female,3,True
31,1408924800,at first glance this product looks cheap. that...,feels cheap is cheap but absolutely works.,"Arts, Crafts & Sewing",Pet Supplies,Snaps,at first glance this product look cheap . tha...,feel cheap is cheap but absolutely work,109,7,4,4,device snap direction real,the snap device feel cheap in your hand but it...,glance product bag wallet small fabric makeup ...,snap device real direction,4,True
40,1390176000,the pliers are easy to use and really do not t...,easy to use snaps hold well,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the plier are easy to use and really do not ta...,easy to use snap hold well,499,6,5,105,"size snap size snap,size common,size most comm...",prior to use these plier i look online for som...,plier force snap hold open process size online...,bit allowance larger fit most snap manner plea...,16,True


# TextRank_keywords 資料整理

In [12]:
from summa import keywords as TextRank
from summa.summarizer import summarize
def textrank_keys(text):
    keywords1 = list()
    for words in TextRank.keywords(text).split('\n'):
        keywords1.extend(words.split(" "))
    keywords1 = set(keywords1)    
    
    return " ".join(list(keywords1))

def textrank_summ_keys(text): 
    keywords2 = list()
    for words in summarize(text, words=8).split('\n'):
        keywords2.extend(words.split(" "))
    keywords2 = set(keywords2)
    
    return " ".join(list(keywords2))

# flit_key_df['TextRank_keywords'] = flit_key_df['lemm_review'].apply(textrank_to_words)
# flit_key_df['TextRank_keywords_num'] = flit_key_df['TextRank_keywords'].apply(calc_num)
flit_key_df['TextRank_keywords'] = flit_key_df['FOP_keywords'] 
flit_key_df['TextRank_keywords_num'] = flit_key_df['FOP_keywords_num'] 
flit_key_df.loc[:,'TextRank_keywords'] = ''
flit_key_df.loc[:,'TextRank_keywords_num'] = 0
flit_key_df.head()



,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,FOP_keywords,FOP_keywords_num,Cheat,TextRank_keywords,TextRank_keywords_num
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck it tear easily and it is my f...,this ribbon suck it tear easily and it is my f...,106,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...,tie tight,2,True,,0
8,1390953600,this is a good product for the price. the wire...,good product for price.,"Arts, Crafts & Sewing",Pet Supplies,Wire,this is a good product for the price . the wi...,good product for price,115,4,4,13,"product good process really manual,process man...",this is a good product for the price .\nalso t...,price product arrive weather wire spool box pr...,american guage really disclaimer manual standa...,13,True,,0
26,1453593600,the snaps work well. they are pretty sticky at...,they are pretty sticky at first and require so...,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the snap work well . they are pretty sticky a...,they are pretty sticky at first and require so...,63,12,3,2,"snap white,snap female",problem though 2 of my pack of 10 white snap h...,snap require snap include female pack snap whi...,white snap female,3,True,,0
31,1408924800,at first glance this product looks cheap. that...,feels cheap is cheap but absolutely works.,"Arts, Crafts & Sewing",Pet Supplies,Snaps,at first glance this product look cheap . tha...,feel cheap is cheap but absolutely work,109,7,4,4,device snap direction real,the snap device feel cheap in your hand but it...,glance product bag wallet small fabric makeup ...,snap device real direction,4,True,,0
40,1390176000,the pliers are easy to use and really do not t...,easy to use snaps hold well,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the plier are easy to use and really do not ta...,easy to use snap hold well,499,6,5,105,"size snap size snap,size common,size most comm...",prior to use these plier i look online for som...,plier force snap hold open process size online...,bit allowance larger fit most snap manner plea...,16,True,,0


In [13]:
from tqdm import tqdm
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        TextRank_keywords = textrank_keys(row['lemm_review'])
    #     TextRank_keywords = textrank_summ_keys(row['lemm_review'])  
        num = calc_keyword_num(TextRank_keywords)
        flit_key_df.loc[i,'TextRank_keywords'] = TextRank_keywords
        flit_key_df.loc[i,'TextRank_keywords_num'] = num
        pbar.update(1)
        
flit_key_df.head()

100%|██████████| 104632/104632 [26:58<00:00, 64.64it/s]


,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,FOP_keywords,FOP_keywords_num,Cheat,TextRank_keywords,TextRank_keywords_num
4,1406419200,this ribbon sucks it tears easily and it is my...,this ribbon sucks it tears easily and it is my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck it tear easily and it is my f...,this ribbon suck it tear easily and it is my f...,106,11,1,3,tie tight,it rip when i try to tie in a bow and the bow ...,ribbon satin read tear satin plastic tie tight...,tie tight,2,True,real mays ribbon,3
8,1390953600,this is a good product for the price. the wire...,good product for price.,"Arts, Crafts & Sewing",Pet Supplies,Wire,this is a good product for the price . the wi...,good product for price,115,4,4,13,"product good process really manual,process man...",this is a good product for the price .\nalso t...,price product arrive weather wire spool box pr...,american guage really disclaimer manual standa...,13,True,brand awg good wire product,5
26,1453593600,the snaps work well. they are pretty sticky at...,they are pretty sticky at first and require so...,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the snap work well . they are pretty sticky a...,they are pretty sticky at first and require so...,63,12,3,2,"snap white,snap female",problem though 2 of my pack of 10 white snap h...,snap require snap include female pack snap whi...,white snap female,3,True,snap use,2
31,1408924800,at first glance this product looks cheap. that...,feels cheap is cheap but absolutely works.,"Arts, Crafts & Sewing",Pet Supplies,Snaps,at first glance this product look cheap . tha...,feel cheap is cheap but absolutely work,109,7,4,4,device snap direction real,the snap device feel cheap in your hand but it...,glance product bag wallet small fabric makeup ...,snap device real direction,4,True,snap look small makeup,4
40,1390176000,the pliers are easy to use and really do not t...,easy to use snaps hold well,"Arts, Crafts & Sewing",Pet Supplies,Snaps,the plier are easy to use and really do not ta...,easy to use snap hold well,499,6,5,105,"size snap size snap,size common,size most comm...",prior to use these plier i look online for som...,plier force snap hold open process size online...,bit allowance larger fit most snap manner plea...,16,True,make issue snap push try life help easier succ...,14


In [14]:
from matplotlib import pyplot as plt
if not os.path.exists('XLSX/statistic'):
    os.makedirs('XLSX/statistic')
with open('XLSX/statistic/%s_info.txt'%(main_cat),'w') as f:
    max_rev_len = flit_key_df['lemm_review_len'].max()
    min_rev_len = flit_key_df['lemm_review_len'].min()
    mean_rev_len = flit_key_df['lemm_review_len'].mean()
    median_rev_len = flit_key_df['lemm_review_len'].median()

    f.write('max_rev_len :%s \n'%(max_rev_len))
    f.write('min_rev_len :%s \n'%(min_rev_len))
    f.write('mean_rev_len :%s \n'%(mean_rev_len))
    f.write('median_rev_len :%s \n'%(median_rev_len))
    
    f.write('\n\n\n')
    max_summary_len = flit_key_df['lemm_summary_len'].max()
    min_summary_len = flit_key_df['lemm_summary_len'].min()
    mean_summary_len = flit_key_df['lemm_summary_len'].mean()
    median_summary_len = flit_key_df['lemm_summary_len'].median()

    f.write('max_summary_len :%s \n'%(max_summary_len))
    f.write('min_summary_len :%s \n'%(min_summary_len))
    f.write('mean_summary_len :%s \n'%(mean_summary_len))
    f.write('median_summary_len :%s \n'%(median_summary_len))
    
    f.write('\n\n\n')
    max_FOP_keywords_num = flit_key_df['FOP_keywords_num'].max()
    min_FOP_keywords_num = flit_key_df['FOP_keywords_num'].min()
    mean_FOP_keywords_num = flit_key_df['FOP_keywords_num'].mean()
    median_FOP_keywords_num = flit_key_df['FOP_keywords_num'].median()

    f.write('max_FOP_keywords_num :%s \n'%(max_FOP_keywords_num))
    f.write('min_FOP_keywords_num :%s \n'%(min_FOP_keywords_num))
    f.write('mean_FOP_keywords_num :%s \n'%(mean_FOP_keywords_num))
    f.write('median_FOP_keywords_num :%s \n'%(median_FOP_keywords_num))
    
    f.write('\n\n\n')
    max_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].max()
    min_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].min()
    mean_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].mean()
    median_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].median()

    f.write('max_TextRank_keywords_num :%s \n'%(max_TextRank_keywords_num))
    f.write('min_TextRank_keywords_num :%s \n'%(min_TextRank_keywords_num))
    f.write('mean_TextRank_keywords_num :%s \n'%(mean_TextRank_keywords_num))
    f.write('median_TextRank_keywords_num :%s \n'%(median_TextRank_keywords_num))

    

# plt.xlim(xmax = mean_rev_len)
# plt.ylim(ymax = flit_key_df['lemm_review_len'].value_counts().max())

flit_key_df['lemm_review_len'].value_counts().hist()
plt.savefig('XLSX/statistic/review_len_%s.png'%(main_cat))
plt.show()
plt.close()

# plt.xlim(xmax = max_summary_len)
# plt.ylim(ymax = flit_key_df['lemm_summary_len'].value_counts().max())
flit_key_df['lemm_summary_len'].value_counts().hist()
plt.savefig('XLSX/statistic/summary_len_%s.png'%(main_cat))
plt.show()
plt.close()

# plt.xlim(xmax = mean_keyword_num)
flit_key_df['FOP_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/FOP_keywords_num_%s.png'%(main_cat))
plt.show()
plt.close()

flit_key_df['TextRank_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/TextRank_keywords_num_%s.png'%(main_cat))
plt.show()
plt.close()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# 製作record bin檔

In [15]:
import shutil
if os.path.exists('bin'):
    shutil.rmtree('/bin', ignore_errors=True)

if not os.path.exists('bin/main_cat/chunked'):
    os.makedirs('bin/main_cat/chunked')

makevocab = True
if makevocab:
    vocab_counter = collections.Counter()
    
# train_file
flit_key_train_df = flit_key_df.iloc[:int(amount*0.6)]

# test_file
flit_key_test_df = flit_key_df.iloc[int(amount*0.6)+1:int(amount*0.8)]

# vald_file
flit_key_valid_df = flit_key_df.iloc[int(amount*0.8)+1:]
sentence_start = "<s>"
sentence_end = "</s>"


def xlsx2bin(set_name,df):
    sents = []
    with open("bin/main_cat/%s.bin"%(set_name), 'wb') as file:
        i = 0
        for idx in tqdm(range(len(df))):
            series = df.iloc[idx]
            data_dict = series.to_dict()
            review_ID , big_categories , small_categories , \
            review , lemm_review , summary , lemm_summary , FOP_keywords ,TextRank_keywords = \
            data_dict['review_ID'],data_dict['big_categories'],data_dict['small_categories'],data_dict['review'],data_dict['lemm_review'], \
            data_dict['summary'],data_dict['lemm_summary'],data_dict['FOP_keywords'] ,data_dict['TextRank_keywords']
#             print(FOP_keywords)

            # save Embedding/word2Vec calculate sents
#             for sent in nltk.sent_tokenize(lemm_review):
#                 sent = sent.replace("." ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

#             for sent in nltk.sent_tokenize(lemm_summary):
#                 sent = sent.replace(sentence_start ,"").replace(sentence_end ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

            lemm_review = lemm_review.replace("\n","")
            lemm_summary = lemm_summary.replace("\n","").replace("."," ")
            lemm_summary = sentence_start + ' '+ lemm_summary + ' ' + sentence_end
#             print(lemm_summary)
            # Write to tf.Example
            tf_example = example_pb2.Example()
            try:
                tf_example.features.feature['orign_review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(review, encoding='utf-8')])

                tf_example.features.feature['orign_summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(summary, encoding='utf-8')])
                
                tf_example.features.feature['review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_review, encoding='utf-8')])

                tf_example.features.feature['summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_summary, encoding='utf-8')]) 
        
                tf_example.features.feature['FOP_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(FOP_keywords, encoding='utf-8')]) 
            
                tf_example.features.feature['TextRank_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(TextRank_keywords, encoding='utf-8')]) 

                tf_example_str = tf_example.SerializeToString()
                str_len = len(tf_example_str)  
                file.write(struct.pack('q', str_len))
                file.write(struct.pack('%ds' % str_len, tf_example_str))
            except Exception as e:
#                 print(e)
                pass
    print(" %s finished... "%(file.name))
    return sents
    
    
sents1 = xlsx2bin('train',flit_key_train_df)
sents2 = xlsx2bin('test',flit_key_test_df)
sents3 = xlsx2bin('valid',flit_key_valid_df)

  1%|▏         | 312/20925 [00:00<00:06, 3119.40it/s]

 bin/main_cat/train.bin finished... 


  2%|▏         | 404/20926 [00:00<00:05, 4032.44it/s]

 bin/main_cat/test.bin finished... 


100%|██████████| 20926/20926 [00:05<00:00, 3793.51it/s]


 bin/main_cat/valid.bin finished... 


In [16]:
with open("bin/main_cat/bin-info.txt",'w',encoding='utf-8') as f :
    f.write("train : %s\n"%(len(flit_key_train_df)))
    f.write("test : %s\n"%(len(flit_key_test_df)))
    f.write("valid : %s\n"%(len(flit_key_valid_df)))

# 分割record bin檔(1000為單位)

In [17]:
def chunk_file(set_name, chunks_dir):
    in_file = 'bin/main_cat/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
#         chunk_fname = os.path.join('bin', '/%s/%s_%03d.bin' % (chunks_dir,set_name, chunk))  # new chunk
        chunk_fname = '%s/%s/%s_%03d.bin' % (chunks_dir,set_name,set_name, chunk)
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all(chunks_dir = 'bin/main_cat/chunked'):
    # Make a dir to hold the chunks
    
    # Chunk the data
    for set_name in ['train', 'valid', 'test']:
        if not os.path.isdir(os.path.join(chunks_dir,set_name)):
            os.mkdir(os.path.join(chunks_dir,set_name))
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name, chunks_dir)
    print("Saved chunked data in %s" % chunks_dir)
    
chunk_all()

Splitting train data into chunks...
Splitting valid data into chunks...
Splitting test data into chunks...
Saved chunked data in bin/main_cat/chunked


In [18]:
def main_valid():
    #Performing rouge evaluation on 1.9 lakh sentences takes lot of time. So, create mini validation set & test set by borrowing 15k samples each from these 1.9 lakh sentences
    bin_valid_chuncks = os.listdir('bin/main_cat/chunked/valid/')
    bin_valid_chuncks.sort()
    if not os.path.exists('bin/main_cat/chunked/main_valid'):
        os.makedirs('bin/main_cat/chunked/main_valid')
        
    samples = random.sample(set(bin_valid_chuncks[:-1]), 2)      #Exclude last bin file; contains only 9k sentences
    valid_chunk, test_chunk = samples[0], samples[1]
    shutil.copyfile(os.path.join('bin/main_cat/chunked/valid', valid_chunk), os.path.join("bin/main_cat/chunked/main_valid", "valid_00.bin"))
    shutil.copyfile(os.path.join('bin/main_cat/chunked/valid', test_chunk), os.path.join("bin/main_cat/chunked/main_valid", "test_00.bin"))
main_valid()

# Embedding/word2Vec

In [19]:
sentences = [] # total sentence
for idx in tqdm(range(len(orign_key_df))):
    series = orign_key_df.iloc[idx]
    data_dict = series.to_dict()
    lemm_review_sents , lemm_summary  = data_dict['lemm_review'],data_dict['lemm_summary'] 
    lemm_review_sents = eval(lemm_review_sents)
    for sent in lemm_review_sents:
        sent_tokens = sent.split(" ")
#         tokens = [str(token) for token in sent.split() if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
        tokens = [str(token) for token in sent.split() if (" " not in str(token))]
        sentences.append(tokens)     
    sentences.append(lemm_summary.split(" "))
print('word2Vec training sentence finished...')

100%|██████████| 547381/547381 [02:12<00:00, 4132.16it/s]

word2Vec training sentence finished...


In [20]:
# sentences = sents1 + sents2 + sents3

In [21]:
# 引入 word2vec
from gensim.models import word2vec
from glob import glob
import sys

import gensim
import torch
import torch.nn as nn
import torchsnooper
import os
import numpy as np

# 引入日志配置
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

vocab_count = 50000
# write vocab to file
if not os.path.exists('Embedding/main_cat/word2Vec'):
    os.makedirs('Embedding/main_cat/word2Vec')

In [22]:
if not os.path.exists("Embedding/main_cat/word2Vec/word2Vec.300d.txt"):

    w2vec = word2vec.Word2Vec(sentences, size=300, min_count=1,max_vocab_size=None,iter=100,
                              sorted_vocab=1,max_final_vocab=vocab_count)    

    w2vec.wv.save_word2vec_format('Embedding/main_cat/word2Vec/word2Vec.300d.txt', binary=False)

    #保存模型，供日後使用
    # w2vec.save("Embedding/word2Vec/word2vec.model")   


2020-01-08 06:59:41,298 : INFO : collecting all words and their counts
2020-01-08 06:59:41,299 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-08 06:59:41,328 : INFO : PROGRESS: at sentence #10000, processed 175051 words, keeping 6644 word types
2020-01-08 06:59:41,351 : INFO : PROGRESS: at sentence #20000, processed 328897 words, keeping 10007 word types
2020-01-08 06:59:41,374 : INFO : PROGRESS: at sentence #30000, processed 486564 words, keeping 12690 word types
2020-01-08 06:59:41,418 : INFO : PROGRESS: at sentence #40000, processed 643196 words, keeping 14669 word types
2020-01-08 06:59:41,440 : INFO : PROGRESS: at sentence #50000, processed 798630 words, keeping 16589 word types
2020-01-08 06:59:41,462 : INFO : PROGRESS: at sentence #60000, processed 948640 words, keeping 18156 word types
2020-01-08 06:59:41,488 : INFO : PROGRESS: at sentence #70000, processed 1117492 words, keeping 19704 word types
2020-01-08 06:59:41,521 : INFO : PROGRESS: at 

In [23]:
#模型讀取方式
# model = word2vec.Word2Vec.load("Embedding/word2Vec/word2vec.model")

wvmodel = gensim.models.KeyedVectors.load_word2vec_format(
    'Embedding/main_cat/word2Vec/word2Vec.300d.txt', binary=False, encoding='utf-8')

wvmodel.most_similar(u"player", topn=10)
# wvmodel.most_similar(['dvd','player','changer','machine','video'], topn=20)

2020-01-08 08:29:02,557 : INFO : loading projection weights from Embedding/main_cat/word2Vec/word2Vec.300d.txt
2020-01-08 08:29:12,569 : INFO : loaded (45605, 300) matrix from Embedding/main_cat/word2Vec/word2Vec.300d.txt
2020-01-08 08:29:12,569 : INFO : precomputing L2-norms of word weight vectors


[('chewer', 0.4208277463912964),
 ('chewer.', 0.40346163511276245),
 ('chewer.but', 0.35819292068481445),
 ('soccer', 0.34019553661346436),
 ('tugger', 0.33368951082229614),
 ('goalie', 0.3328365683555603),
 ('game', 0.32683518528938293),
 ('album', 0.3221092224121094),
 ('puller', 0.3198249042034149),
 ('play', 0.31764525175094604)]

In [24]:
vocab_file = "Embedding/main_cat/word2Vec/word.vocab"

if not os.path.exists(vocab_file):
    vocab_count = len(wvmodel.wv.index2entity)    

    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word in wvmodel.wv.index2entity[:vocab_count]:
            # print(word, w2vec.wv.vocab[word].count)
            writer.write(word + ' ' + str(wvmodel.wv.vocab[word].count) + '\n') # Output vocab count
    print("Finished writing vocab file")

/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


Writing vocab file...
Finished writing vocab file


In [25]:
word = wvmodel.wv.index2entity[25]
vector = wvmodel.wv.vectors[25]
print(word)
# print(vector)

are


/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [26]:
import torch
from data_util.data import Vocab
vocab_size = len(wvmodel.vocab) + 1


vocab = Vocab('Embedding/main_cat/word2Vec/word.vocab', vocab_size)

embed_size = 300
weight = torch.zeros(vocab_size, embed_size)

for i in range(len(vocab._id_to_word.keys())):
    try:
        vocab_word = vocab._id_to_word[i+4]
        w2vec_word = w2vec.wv.index2entity[i]
    except Exception as e :
        continue
    if i + 4 > vocab_size: break
#     print(vocab_word,w2vec_word)
    weight[i+4, :] = torch.from_numpy(w2vec.wv.vectors[i])
        
embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding

Embedding(45606, 300)

In [27]:
vocab.word2id('the')

5

# Embedding/glove

In [28]:
from glove import Glove
from glove import Corpus

vocab_count = 50000
# write vocab to file
if not os.path.exists('Embedding/main_cat/glove'):
    os.makedirs('Embedding/main_cat/glove')

In [29]:
if not os.path.exists("Embedding/main_cat/glove/glove.model"):

    corpus_model = Corpus()
    corpus_model.fit(sentences, window=10)
    #corpus_model.save('corpus.model')
    print('Dict size: %s' % len(corpus_model.dictionary))
    print('Collocations: %s' % corpus_model.matrix.nnz)
    
    glove = Glove(no_components=300, learning_rate=0.05)
    glove.fit(corpus_model.matrix, epochs=100,
              no_threads=10, verbose=True)
    glove.add_dictionary(corpus_model.dictionary)
    
    glove.save('Embedding/main_cat/glove/glove.model') # 存模型
    corpus_model.save('Embedding/main_cat/glove/corpus.model') # 存字典


glove = Glove.load('Embedding/main_cat/glove/glove.model')
corpus_model = Corpus.load('Embedding/main_cat/glove/corpus.model')

Dict size: 170691
Collocations: 18381219
Performing 100 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [30]:
vocab_file = "Embedding/main_cat/glove/word.vocab"

if not os.path.exists(vocab_file):
#     vocab_count = len(glove.dictionary)    
    vocab_count = 0
    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word,idx in glove.dictionary.items():
            if word in vocab._word_to_id.keys():
                vocab_count += 1
                writer.write(word + ' ' + str(idx) + '\n') # Output vocab count
    print("Finished writing vocab file %s" %(vocab_count))

Writing vocab file...
Finished writing vocab file 45602


In [31]:
glove.word_vectors[glove.dictionary['.']].shape
# vocab._word_to_id.keys()
# len(glove.dictionary)

(300,)

In [32]:
vocab_size = len(open('Embedding/main_cat/glove/word.vocab').readlines())
print(vocab_size)

vocab = Vocab('Embedding/main_cat/glove/word.vocab', vocab_size)
embed_size = 300
weight = torch.zeros(vocab_size, embed_size)

for word,idx in glove.dictionary.items():
    if word in vocab._word_to_id.keys():
        wid = vocab.word2id(word) 
        vector = np.asarray(glove.word_vectors[glove.dictionary[word]], "float32")
        weight[wid, :] = torch.from_numpy(vector)

embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding 

45602


Embedding(45602, 300)

# Embedding/Bert

In [33]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
model.embeddings.word_embeddings


# vocab = Vocab('Embedding/word2Vec/word2Vec.vocab', vocab_size)

# embed_size = 300
# weight = torch.zeros(vocab_size, embed_size)


# embedding = torch.nn.Embedding.from_pretrained(weight)
# # requires_grad指定是否在训练过程中对词向量的权重进行微调
# embedding.weight.requires_grad = True
# embedding        

2020-01-08 09:12:13,439 : INFO : Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-01-08 09:12:14,418 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/eagleuser/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2020-01-08 09:12:14,421 : INFO : extracting archive file /home/eagleuser/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpu0togylk
2020-01-08 09:12:17,270 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layer

Embedding(30522, 768, padding_idx=0)